# Sprint Seq2Seq

## 【問題1】機械翻訳の実行とコードリーディング

In [2]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [6]:
cd /content/drive/My Drive/DIC/dataset

/content/drive/My Drive/DIC/dataset


In [7]:
# 英仏文のデータをgoogleドライブにダウンロード
!wget http://www.manythings.org/anki/fra-eng.zip
# 解凍
!unzip fra-eng.zip

--2020-10-09 02:23:35--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 172.67.173.198, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6073837 (5.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   5.79M  1.78MB/s    in 3.3s    

2020-10-09 02:23:38 (1.78 MB/s) - ‘fra-eng.zip’ saved [6073837/6073837]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


* 短い英文から短い仏文への機械翻訳を行う
* LSTMが入力値を2通りのベクターに変換する。
* LSTMデコーダーは目的シーケンスを同じにするため、１ステップ先で相殺させる訓練を行う。
  * これを「強制訓練」と述べている。
  * よって、目的変数の時系列tを入力値として、t+1を出力する。
* 未知のシーケンスを入力した場合。
  1. 入力値をstate_vectorsに符号化する。
  2. size_1のtaget_sequenceを実行
  3. state_vectorsとtarget_sequenceを複合化に放り込んで、次の語の推測に用いる。
  4. argmaxを用いて次の語のサンプルを推測
  5. 推測したサンプルをtarget_sequenceに追加
  6. 以上を繰り返す。

In [17]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

* 入力データをベクター化
  * fra.txtを読み込む。
  * txt内の各行を行毎に正規表現
    * input_texts, target_textsに代入する。
    * input_texts, target_textsから各語をinput_characters, target_charactersに追記する。
    * input_characterの要素数がnum_encoder_tokens
    * taget_characterの要素数がnum_decoder_tokens
    

In [18]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


* encoder_inputsをLSTMで処理する。
  * state_h, state_cのみを残す。

* decoder_inputsをnum_decoder_tokensの要素数を列数として形成する。

In [ ]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

* encoder_input_dataとdecoder_input_dataをdecoder_target_dataとしてmodel化する。
* モデルの訓練を行い、保存する。

In [ ]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

* サンプリング
  1. 入力値を符号化、複合用の初期値とする。
  2. その初期値を用いて、複合化を一度実施する。
    * start_of_sequenceのtokenを目的変数とする。
    * 出力値が次のtokenとなる。
  3. 以上を繰り返す。

In [21]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Epoch 1/100
125/125 [==============================] - 54s 432ms/step - loss: 1.1765 - accuracy: 0.7251 - val_loss: 1.0373 - val_accuracy: 0.7118
Epoch 2/100
125/125 [==============================] - 53s 427ms/step - loss: 0.8524 - accuracy: 0.7695 - val_loss: 0.8353 - val_accuracy: 0.7684
Epoch 3/100
125/125 [==============================] - 52s 417ms/step - loss: 0.6789 - accuracy: 0.8078 - val_loss: 0.7219 - val_accuracy: 0.7880
Epoch 4/100
125/125 [==============================] - 52s 417ms/step - loss: 0.5931 - accuracy: 0.8276 - val_loss: 0.6494 - val_accuracy: 0.8099
Epoch 5/100
125/125 [==============================] - 52s 415ms/step - loss: 0.5431 - accuracy: 0.8418 - val_loss: 0.6058 - val_accuracy: 0.8210
Epoch 6/100
125/125 [==============================] - 53s 423ms/step - loss: 0.5057 - accuracy: 0.8516 - val_loss: 0.5839 - val_accuracy: 0.8282
Epoch 7/100
125/125 [==============================] - 52s 416ms/step - loss: 0.4774 - accuracy: 0.8593 - val_loss: 0.5529 -

## 【問題2】イメージキャプショニングの学習済みモデルの実行

In [1]:
cd /content/drive/My Drive/DIC/Git

/content/drive/My Drive/DIC/Git


In [2]:
!git clone https://github.com/pdollar/coco.git

Cloning into 'coco'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 5.74 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [3]:
cd ./coco/PythonAPI/

/content/drive/My Drive/DIC/Git/coco/PythonAPI


In [4]:
!make
!python setup.py build
!python setup.py install

python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/My Drive/DIC/Git/coco/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/pycocotools
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.6/dist-packages/numpy/core/include -I../common -I/usr/include/python3.6m -c ../common/maskApi.c -o build/temp.linux-x86_64-3.6/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
../common/maskApi.c: In function ‘rleDecode’:


In [6]:
cd ../../

/content/drive/My Drive/DIC/Git


In [11]:
!git clone https://github.com/yunjey/pytorch-tutorial.git

Cloning into 'pytorch-tutorial'...
remote: Enumerating objects: 917, done.
remote: Total 917 (delta 0), reused 0 (delta 0), pack-reused 917
Receiving objects: 100% (917/917), 12.80 MiB | 2.64 MiB/s, done.
Resolving deltas: 100% (491/491), done.


In [7]:
cd pytorch-tutorial/tutorials/03-advanced/image_captioning/

/content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [13]:
!pip install -r requirements.txt

In [45]:
cd /content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning/models

/content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning/models


In [46]:
!unzip pretrained_model.zip

Archive:  pretrained_model.zip
  inflating: encoder-5-3000.pkl      
  inflating: decoder-5-3000.pkl      


In [47]:
cd /content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning/data

/content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning/data


In [49]:
!unzip vocap.zip

Archive:  vocap.zip
  inflating: vocab.pkl               


In [51]:
cd /content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning/

/content/drive/My Drive/DIC/Git/pytorch-tutorial/tutorials/03-advanced/image_captioning


In [9]:
!python sample.py --image='png/example.png'

<start> a group of giraffes standing next to each other . <end>


## 【問題3】Kerasで動かしたい場合はどうするかを調査

PyTorchではなく、Kerasで動かしたい場合。
* ONNX (Open Neural Network Exchange)を用いて、簡易的に以下のフォーマット同士の変換が可能である。
  * Pytorch, Tensorflow, Keras, Cafee2, CoreML
  * [参照元](https://medium.com/@santhoshkolloju/convert-your-pytorch-models-to-tensorflow-with-onnx-84c3bdd8d722)
* 今回の場合は、Pytorchモデルを一旦、ONNXフォーマットに変換する。そのコードは以下の通り。

In [ ]:
from transformers import (WEIGHTS_NAME, BertConfig, BertForQuestionAnswering, BertTokenizer)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
model.eval()

inputs = {
    'input_ids':      np.array(inp).reshape(1, 128),                         
    'attention_mask': np.array(mask).reshape(1, 128),
    'token_type_ids': np.array(typeid).reshape(1, 128)
}
torch.onnx.export(model,                                            
                      (inputs['input_ids'],                             
                       inputs['attention_mask'], 
                inputs['token_type_ids'])#sample input to the model, 
                      output_model_path="BERT.onnx",                                
                      opset_version=11,                                 
                      do_constant_folding=True,                         
                      input_names=['input_ids',                         
                                   'input_mask', 
                                   'segment_ids'],
                      output_names=['start', 'end'],                    
                      dynamic_axes={'input_ids': [1], #this means second axis is dynamic       
                                    'input_mask' : [1],
                                    'segment_ids' : [1],
                                    })

* 次にONNXフォーマットをTensorflowに変換する。
  * [こちら](https://github.com/onnx/onnx-tensorflow)のパッケージをインストールして、以下のコマンドを実行する。
  * onnx-tf convert -t bert.onnx -o bert.pb
  * 以上である。